In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
sns.set_style()

In [2]:
data = pd.read_csv('../input/crop-recommendation-dataset/Crop_recommendation.csv')
data.drop(['N', 'P', 'K', 'ph'], axis=1, inplace=True)
data.head()

,temperature,humidity,rainfall,label
0,20.879744,82.002744,202.935536,rice
1,21.770462,80.319644,226.655537,rice
2,23.004459,82.320763,263.964248,rice
3,26.491096,80.158363,242.864034,rice
4,20.130175,81.604873,262.717340,rice


In [3]:
data['label'].unique()[10:] # Second Eleven Label

array(['banana', 'mango', 'grapes', 'watermelon', 'muskmelon', 'apple',
       'orange', 'papaya', 'coconut', 'cotton', 'jute', 'coffee'],
      dtype=object)

## Data Preprocessing

In [4]:
def convert_label(row, label):
    if row['label'] == label:
        row['label'] = 1
    else:
        row['label'] = 0
    return row

def take_200(data, label):
    
    # Data for particular label
    specific_data = data[data['label'] == label]
    
    n = len(specific_data)
    
    # Data for beside particular label
    beside_data = data[data['label'] != label].sample(n)
    
    # Combine specific_data and beside_data
    combined_data = pd.concat([specific_data, beside_data])
    
    # Apply combined_data to convert_label function
    new_data = combined_data.apply(convert_label, label=label, axis=1)
    
    # Reset index and shuffle
    return new_data.sample(frac=1).reset_index(drop=True)

In [5]:
print('First 5 observations of data: \n')
take_200(data, 'rice').head(5)

First 5 observations of data: 



,temperature,humidity,rainfall,label
0,25.282722,80.543728,220.115671,1
1,30.107734,90.345464,75.245220,0
2,24.185532,46.687468,177.337800,0
3,21.019447,82.952217,298.401847,1
4,25.161254,55.254358,40.897328,0


## Model Building

In [6]:
banana_data = take_200(data, 'banana')
banana_data.head()

,temperature,humidity,rainfall,label
0,22.145937,82.859755,60.653817,0
1,25.449262,79.492220,100.661917,1
2,21.723831,80.238990,277.962619,0
3,25.314685,77.917571,72.829021,0
4,25.434805,81.539778,96.478004,1


In [7]:
# Define variables
X = banana_data
y = X.pop('label')

In [8]:
# Input Structure
X.head(1)

,temperature,humidity,rainfall
0,22.145937,82.859755,60.653817


In [9]:
'''
ACCURACY_THRESHOLD = 0.95
VAL_ACCURACY_THRESHOLD = 0.95

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        acc = logs.get('acc')
        val_acc = logs.get('val_acc')
        if(acc > 0.95):
            print("\nReached %2.2f%% accuracy, so stopping training!!" %(ACCURACY_THRESHOLD*100))
            self.model.stop_training = True

callbacks = myCallback()
'''

'\nACCURACY_THRESHOLD = 0.95\nVAL_ACCURACY_THRESHOLD = 0.95\n\nclass myCallback(tf.keras.callbacks.Callback):\n    def on_epoch_end(self, epoch, logs={}):\n        acc = logs.get(\'acc\')\n        val_acc = logs.get(\'val_acc\')\n        if(acc > 0.95):\n            print("\nReached %2.2f%% accuracy, so stopping training!!" %(ACCURACY_THRESHOLD*100))\n            self.model.stop_training = True\n\ncallbacks = myCallback()\n'

In [10]:
from sklearn.model_selection import train_test_split

# Split Training and Testing Dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                    random_state=42)
# Build a model structure
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(16, input_shape=[3], activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=tf.keras.optimizers.Adam(), 
             loss='binary_crossentropy',
             metrics=['accuracy'])

history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100)

2022-05-28 10:06:25.890715: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2022-05-28 10:06:26.052890: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100
5/5 [==============================] - 1s 61ms/step - loss: 2.6405 - accuracy: 0.4938 - val_loss: 1.1057 - val_accuracy: 0.5500
Epoch 2/100
5/5 [==============================] - 0s 10ms/step - loss: 0.9285 - accuracy: 0.4938 - val_loss: 0.8900 - val_accuracy: 0.4750
Epoch 3/100
5/5 [==============================] - 0s 10ms/step - loss: 0.9002 - accuracy: 0.4875 - val_loss: 0.8893 - val_accuracy: 0.4750
Epoch 4/100
5/5 [==============================] - 0s 11ms/step - loss: 0.6694 - accuracy: 0.5125 - val_loss: 0.5961 - val_accuracy: 0.6500
Epoch 5/100
5/5 [==============================] - 0s 11ms/step - loss: 0.5314 - accuracy: 0.7375 - val_loss: 0.6467 - val_accuracy: 0.6250
Epoch 6/100
5/5 [==============================] - 0s 10ms/step - loss: 0.4978 - accuracy: 0.6938 - val_loss: 0.5850 - val_accuracy: 0.7000
Epoch 7/100
5/5 [==============================] - 0s 10ms/step - loss: 0.3957 - accuracy: 0.8750 - val_loss: 0.5313 - val_accuracy: 0.6750
Epoch 8/100
5/5 [===

In [11]:
second_11_labels = data['label'].unique()[10:]

def model(data, labels):
    for label in labels:
        # Initiate data
        crop_data = take_200(data, label)
        
        # Initiate variables
        X = crop_data
        y = X.pop('label')
        
        # Split Training and Testing Dataset
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                            random_state=42)
        
        # Initiate a model structure
        model = tf.keras.models.Sequential([
            tf.keras.layers.Dense(64, input_shape=[3], activation='relu'),
            tf.keras.layers.Dense(64, activation='relu'),
            #tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dense(1, activation='sigmoid')
        ])
        
        model.compile(optimizer=tf.keras.optimizers.Adam(), 
             loss='binary_crossentropy',
             metrics=['accuracy'])
        
        # Fitting the Modek
        history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, verbose=0)
        
        print('{} model => Accuracy: {} & Validation Accuracy: {}\n'.format(label.capitalize(), history.history['accuracy'][-1], 
                                                                          history.history['val_accuracy'][-1]))

In [12]:
model(data, second_11_labels)

Banana model => Accuracy: 0.9750000238418579 & Validation Accuracy: 1.0

Mango model => Accuracy: 1.0 & Validation Accuracy: 1.0

Grapes model => Accuracy: 0.9624999761581421 & Validation Accuracy: 1.0

Watermelon model => Accuracy: 0.96875 & Validation Accuracy: 1.0

Muskmelon model => Accuracy: 1.0 & Validation Accuracy: 1.0

Apple model => Accuracy: 0.925000011920929 & Validation Accuracy: 0.925000011920929

Orange model => Accuracy: 0.8812500238418579 & Validation Accuracy: 0.9750000238418579

Papaya model => Accuracy: 0.7124999761581421 & Validation Accuracy: 0.75

Coconut model => Accuracy: 0.949999988079071 & Validation Accuracy: 0.925000011920929

Cotton model => Accuracy: 0.9624999761581421 & Validation Accuracy: 0.8999999761581421

Jute model => Accuracy: 0.949999988079071 & Validation Accuracy: 0.949999988079071

Coffee model => Accuracy: 0.96875 & Validation Accuracy: 0.9750000238418579

